Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [3]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [4]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [7]:
from six.moves import range

In [32]:
# Build the TensorFlow graph with L2 regularization

batch_size = 128
hidden_nodes = 1024
l2_strength = 5e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1  = tf.Variable(tf.zeros([hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2  = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  relu_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_1 = tf.nn.relu(relu_1)
  logits = tf.matmul(relu_1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Use L2 regularization
  l2_loss = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss += l2_strength*l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)

In [33]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 1911.110718
Minibatch accuracy: 9.4%
Validation accuracy: 20.5%
Minibatch loss at step 500: 126.877388
Minibatch accuracy: 84.4%
Validation accuracy: 82.7%
Minibatch loss at step 1000: 11.048361
Minibatch accuracy: 78.9%
Validation accuracy: 85.2%
Minibatch loss at step 1500: 1.393824
Minibatch accuracy: 92.2%
Validation accuracy: 84.8%
Minibatch loss at step 2000: 0.702677
Minibatch accuracy: 83.6%
Validation accuracy: 84.9%
Minibatch loss at step 2500: 0.623778
Minibatch accuracy: 85.2%
Validation accuracy: 85.0%
Minibatch loss at step 3000: 0.702779
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Test accuracy: 91.4%


The current accuracy (91,4%) is better than the previous with no regularization (88,6%). And the Test Accuracy is much higher than the minibatch/validation accuracy, so we can say the model is generalized.

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [36]:
batches = 5
num_steps = 3001
train_labels_small  = train_labels[0:(batch_size*batches), :]
train_dataset_small = train_dataset[0:(batch_size*batches), :]

print("Total training samples:", train_dataset_small.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Total training samples: 640
Initialized
Minibatch loss at step 0: 1870.241699
Minibatch accuracy: 7.8%
Validation accuracy: 27.3%
Minibatch loss at step 500: 128.591248
Minibatch accuracy: 100.0%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 10.611886
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Minibatch loss at step 1500: 1.065916
Minibatch accuracy: 100.0%
Validation accuracy: 79.1%
Minibatch loss at step 2000: 0.275970
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 2500: 0.202047
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Minibatch loss at step 3000: 0.190743
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Test accuracy: 86.3%


With just 5 batches (640 samples) we can still get a pretty good Test accuracy. The model doesn't seem overfitted to the 5 minibatches because it can still find a general solution.

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [53]:
# Train only with Dropout

batch_size = 128
hidden_nodes = 1024
dropout_keep_prob = 0.7

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob_1 = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1  = tf.Variable(tf.zeros([hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2  = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  relu_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_1 = tf.nn.relu(relu_1)
  
  # Apply dropout to the outputs of the first hidden layer
  drop_1 = tf.nn.dropout(relu_1, keep_prob_1)
    
  logits = tf.matmul(drop_1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob_1 : dropout_keep_prob}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at step 0: 395.956787
Minibatch accuracy: 7.8%
Validation accuracy: 17.0%
Minibatch loss at step 500: 16.826612
Minibatch accuracy: 79.7%
Validation accuracy: 81.1%
Minibatch loss at step 1000: 7.952658
Minibatch accuracy: 78.9%
Validation accuracy: 81.0%
Minibatch loss at step 1500: 5.441921
Minibatch accuracy: 78.9%
Validation accuracy: 80.2%
Minibatch loss at step 2000: 5.713160
Minibatch accuracy: 79.7%
Validation accuracy: 80.4%
Minibatch loss at step 2500: 4.556868
Minibatch accuracy: 80.5%
Validation accuracy: 81.0%
Minibatch loss at step 3000: 7.306358
Minibatch accuracy: 76.6%
Validation accuracy: 81.4%
Test accuracy: 88.1%


The model with dropout doesn't produce a much higher score than the model without dropout. However, we can see that it seems to be generalizing the solution because  `[Test acc.] > [Validation acc.] > [Train acc.]`.

In [54]:
# Extreme overfitting case
batches = 5
num_steps = 3001
train_labels_small  = train_labels[0:(batch_size*batches), :]
train_dataset_small = train_dataset[0:(batch_size*batches), :]

print("Total training samples:", train_dataset_small.shape[0])

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_small.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_small[offset:(offset + batch_size), :]
    batch_labels = train_labels_small[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob_1 : dropout_keep_prob}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Total training samples: 640
Initialized
Minibatch loss at step 0: 424.628662
Minibatch accuracy: 10.2%
Validation accuracy: 33.6%
Minibatch loss at step 500: 2.400055
Minibatch accuracy: 99.2%
Validation accuracy: 78.1%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.7%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 78.0%
Test accuracy: 85.1%


Even the extreme overfitting case produces a general solution.

In [55]:
# Train with Dropout + L2 regularization

batch_size = 128
hidden_nodes = 1024
dropout_keep_prob = 0.7
l2_strength = 5e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob_1 = tf.placeholder(tf.float32)
  
  # Variables.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
  biases_1  = tf.Variable(tf.zeros([hidden_nodes]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
  biases_2  = tf.Variable(tf.zeros([num_labels]))

  
  # Training computation.
  relu_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
  relu_1 = tf.nn.relu(relu_1)
  
  # Apply dropout to the outputs of the first hidden layer
  drop_1 = tf.nn.dropout(relu_1, keep_prob_1)
    
  logits = tf.matmul(drop_1, weights_2) + biases_2
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Use L2 regularization
  l2_loss = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
  loss += l2_strength*l2_loss
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2)
  test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2)


num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob_1 : dropout_keep_prob}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at step 0: 1949.289062
Minibatch accuracy: 4.7%
Validation accuracy: 32.3%
Minibatch loss at step 500: 127.189072
Minibatch accuracy: 80.5%
Validation accuracy: 82.5%
Minibatch loss at step 1000: 11.066168
Minibatch accuracy: 78.9%
Validation accuracy: 85.0%
Minibatch loss at step 1500: 1.479852
Minibatch accuracy: 85.2%
Validation accuracy: 85.1%
Minibatch loss at step 2000: 0.754274
Minibatch accuracy: 82.8%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 0.667811
Minibatch accuracy: 82.8%
Validation accuracy: 85.1%
Minibatch loss at step 3000: 0.747018
Minibatch accuracy: 80.5%
Validation accuracy: 84.6%
Test accuracy: 91.1%


Dropout doesn't seem to affect the results when applied to our model with L2 regularization. MNIST is a large dataset and that probably makes dropout unnecessary.

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [46]:
# Our approach is to build a network similar to the one which got 97,1% test accuracy.
# We also add L2 regularization (no dropout), decaying learning rate and use AdamOptimizer.
# Other changes include the usage of ReLUs instead of tanh and exponential learning
# rate decay instead of linear learning rate decay.

batch_size = 128
num_steps = 412 * train_labels.shape[0] / batch_size # 412 epochs
hidden_nodes_1 = 1024
hidden_nodes_2 = 300
hidden_nodes_3 = 50
l2_strength = 0 # 1e-6 # very weak because we are summing regularizers from many weight matrixes
initial_learning_rate = 5e-6 # smaller because we are using more layers
learning_rate_decay_steps = int(1 * train_labels.shape[0] / batch_size) # 1 epoch

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  global_step = tf.Variable(0)  # count the number of steps taken.
  weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes_1]))
  weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2]))
  weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3]))
  weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, num_labels]))
  biases_1  = tf.Variable(tf.zeros([hidden_nodes_1]))
  biases_2  = tf.Variable(tf.zeros([hidden_nodes_2]))
  biases_3  = tf.Variable(tf.zeros([hidden_nodes_3]))
  biases_4  = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_1) + biases_1)
  hidden_2 = tf.nn.relu(tf.matmul(hidden_1, weights_2) + biases_2)
  hidden_3 = tf.nn.relu(tf.matmul(hidden_2, weights_3) + biases_3)

  logits = tf.matmul(hidden_3, weights_4) + biases_4
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))

  # Use L2 regularization
  l2_loss = (tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) +
             tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4))
  loss += l2_strength * l2_loss
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(initial_learning_rate, global_step, learning_rate_decay_steps, 0.96)
  optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3), weights_4) + biases_4)
  test_prediction  = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + biases_1), weights_2) + biases_2), weights_3) + biases_3), weights_4) + biases_4)

In [47]:
with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  print("Total steps:", num_steps)
  print("Steps per epoch:", train_labels.shape[0] / batch_size)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Learning rate:", lr)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Total steps: 643750
Steps per epoch: 1562
Learning rate: 5e-06
Minibatch loss at step 0: 24874.755859
Minibatch accuracy: 11.7%
Validation accuracy: 13.6%
Learning rate: 4.93509e-06
Minibatch loss at step 500: 17314.750000
Minibatch accuracy: 14.1%
Validation accuracy: 13.6%
Learning rate: 4.87102e-06
Minibatch loss at step 1000: 12671.798828
Minibatch accuracy: 10.2%
Validation accuracy: 14.3%
Learning rate: 4.80778e-06
Minibatch loss at step 1500: 8809.632812
Minibatch accuracy: 21.1%
Validation accuracy: 17.9%
Learning rate: 4.74537e-06
Minibatch loss at step 2000: 6990.116699
Minibatch accuracy: 20.3%
Validation accuracy: 22.1%
Learning rate: 4.68376e-06
Minibatch loss at step 2500: 5583.059570
Minibatch accuracy: 26.6%
Validation accuracy: 26.4%
Learning rate: 4.62296e-06
Minibatch loss at step 3000: 4710.575195
Minibatch accuracy: 29.7%
Validation accuracy: 30.6%
Learning rate: 4.56294e-06
Minibatch loss at step 3500: 4437.015137
Minibatch accuracy: 33.6%
Validation a

KeyboardInterrupt: 

We have tested dozens of network architectures and hyperparameters. Unfortunately none of them reached a better test accuracy than the 1 hidden layer network with L2 regularization.

So we have just implemented a network based on the "97,1% test accuracy model" with some hyperparameters chosen by us.

I think we have a problem with unstable gradients, as explained at http://neuralnetworksanddeeplearning.com/chap5.html and visualized at https://cs224d.stanford.edu/notebooks/vanishing_grad_example.html.

Also worth noting. We have found several cases where the loss decreases, but the accuracy doesn't get better.